PROYECTO FINAL

In [ ]:
import os
import requests
import time
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("AEMET_API_KEY")
CSV_ESTACIONES = "data/estaciones_filtradas.csv"
ARCHIVO_SALIDA = "data/temperaturas_historicas_ampliadas.csv"

# Definir los 4 rangos de fechas para cubrir 2 años
FECHAS = [
    ("2023-05-29T00:00:00UTC", "2023-11-28T00:00:00UTC"),
    ("2023-11-29T00:00:00UTC", "2024-05-28T00:00:00UTC"),
    ("2024-05-29T00:00:00UTC", "2024-11-28T00:00:00UTC"),
    ("2024-11-29T00:00:00UTC", "2025-05-28T00:00:00UTC")
]

# Si ya existe el archivo, cargamos los idema descargados
if os.path.exists(ARCHIVO_SALIDA):
    datos_existentes = pd.read_csv(ARCHIVO_SALIDA)
    estaciones_descargadas = set(datos_existentes["idema"].unique())
else:
    estaciones_descargadas = set()

# Leer estaciones
estaciones = pd.read_csv(CSV_ESTACIONES)

# Lista para guardar todos los datos
todas_las_filas = []

print("Cargando estaciones...")

for _, fila in estaciones.iterrows():
    codigo = fila["indicativo"]
    nombre = fila["nombre"]

    if codigo in estaciones_descargadas:
        continue

    print(f"Procesando estación: {codigo} - {nombre}")

    datos_estacion = []
    for fecha_inicio, fecha_fin in FECHAS:
        url_meta = (
            f"https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/"
            f"datos/fechaini/{fecha_inicio}/fechafin/{fecha_fin}/estacion/{codigo}"
        )

        try:
            respuesta_meta = requests.get(url_meta, params={"api_key": API_KEY})
            if respuesta_meta.status_code != 200:
                continue

            url_datos = respuesta_meta.json().get("datos")
            if not url_datos:
                continue

            respuesta_datos = requests.get(url_datos)
            if respuesta_datos.status_code != 200:
                continue

            datos_json = respuesta_datos.json()
            for fila in datos_json:
                fila["idema"] = codigo
                fila["nombre_estacion"] = nombre
                datos_estacion.append(fila)

            time.sleep(1.5)  # Espera para no saturar la API

        except Exception:
            continue

    todas_las_filas.extend(datos_estacion)

# Convertir a DataFrame y guardar
if todas_las_filas:
    df = pd.DataFrame(todas_las_filas)

    if os.path.exists(ARCHIVO_SALIDA):
        df.to_csv(ARCHIVO_SALIDA, mode="a", index=False, header=False)
    else:
        df.to_csv(ARCHIVO_SALIDA, index=False)

    print("Los datos se han guardado correctamente.")
else:
    print("No se obtuvieron datos nuevos.")